# Introduction

This IPython notebook represents my written report and source code for COMPSCI 369 Assignment 3. To generate a PDF version of this report, simply type `rake` at the command line. The generated report can be found at `report.pdf`. Please note that you will need the following software installed.

* Graphviz
* IPython 3
* \LaTeX
* matplotlib
* NumPy
* pandoc
* PDFtk
* R
* Rake

In [ ]:
from IPython.display import display_pdf, set_matplotlib_formats
import itertools as it
from matplotlib import pyplot as plt
%matplotlib inline
set_matplotlib_formats('pdf')
import numpy as np
import subprocess
from Tree import Tree
from Node import Node

# Question 1

Consider the following HMM for secondary structure in protein sequences. The secondary structure at a residue is either an $\alpha$-helix, $\beta$-strand, or a loop, represented by the states

In [ ]:
STATES = ('H', 'E', 'T')

The state transitions are described by the following diagram. The initial state for the sequence is distributed uniformly.

In [ ]:
A = {
    'H': {'H': 15/16, 'E': 3/160, 'T': 7/160},
    'E': {'H': 1/15, 'E': 5/6, 'T': 1/10},
    'T': {'H': 1/8, 'E': 1/8, 'T': 3/4},
}

edges = ('{} -> {} [label = {:.5g}];'.format(x, y, A[x][y])
         for x, y in it.product(STATES, repeat=2))
hmm = 'digraph {{{}}}'.format(''.join(edges)).encode()
display_pdf(subprocess.check_output(['dot', '-Tpdf'], input=hmm), raw=True)

A state may emit a hydrophobic, hydrophilic, or neutral amino acid.

In [ ]:
SYMBOLS = ('B', 'I', 'N')

The emission probabilities for each state are given by the following table.

In [ ]:
E = {
    'H': {'B': 0.3, 'I': 0.5, 'N': 0.2},
    'E': {'B': 0.55, 'I': 0.15, 'N': 0.3},
    'T': {'B': 0.1, 'I': 0.4, 'N': 0.5},
}

|       |**B**|**I**|**N**|
|:-----:|:---:|:---:|:---:|
| **H** | 0.3 | 0.5 | 0.2 |
| **E** | 0.55| 0.15| 0.3 |
| **T** | 0.1 | 0.4 | 0.5 |

Here, we simulate a sequence of length 200 under the HMM.

In [ ]:
def d2l(d, K):
    return [d[k] for k in K]

def simulate_hmm(length):
    states = []
    symbols = []
    if length > 0:
        states.append(np.random.choice(STATES))
        symbols.append(np.random.choice(SYMBOLS, p=d2l(E[states[-1]], SYMBOLS)))
        for _ in range(1, length):
            states.append(np.random.choice(STATES, p=d2l(A[states[-1]], STATES)))
            symbols.append(np.random.choice(SYMBOLS, p=d2l(E[states[-1]], SYMBOLS)))
    return states, symbols

states, symbols = simulate_hmm(200)
print('\n'.join(','.join(s) for s in np.array_split(states, 5)))
print()
print('\n'.join(','.join(s) for s in np.array_split(symbols, 5)))

Then we calculate the logarithm of the joint probability $\log{\left(P\left(x,\pi\right)\right)}$ of this sequence.

In [ ]:
def joint_logp(states, symbols):
    logp = 0
    if len(states) > 0:
        logp -= np.log2(len(STATES))
        logp += np.log2(E[states[0]][symbols[0]])
        for i in range(1, len(states)):
            logp += np.log2(A[states[i-1]][states[i]])
            logp += np.log2(E[states[i]][symbols[i]])
    return logp

joint_logp(states, symbols)

Given that
$$\pi = H,H,H,H,H,T,T,E,E,E,H,H,H,H,H,H,E,E,E,E,E,E$$
$$x = N,I,N,B,N,I,I,B,N,I,B,B,I,N,B,I,I,N,B,B,N,B$$
then $\log\left(P\left(x,\pi\right)\right)$ is

In [ ]:
pi = ['H','H','H','H','H','T','T','E','E','E','H','H','H','H','H','H','E','E','E','E','E','E']
x = ['N','I','N','B','N','I','I','B','N','I','B','B','I','N','B','I','I','N','B','B','N','B']
joint_logp(pi, x)

The log probability of the simulated symbols is given by the forward algorithm. 

In [ ]:
def log2sum(x):
    x = np.array(x)
    return x[0] + np.log2(np.sum(np.power(2, x - x[0])))

def forward(x):
    logp = np.array([0.0] * len(STATES))
    if len(x) > 0:
        for i, st in enumerate(STATES):
            logp[i] = np.log2(E[st][x[0]]) - np.log2(len(STATES))
        for sy in x[1:]:
            prev = logp
            logp = np.array([0.0] * len(STATES))
            for i, st in enumerate(STATES):
                logp[i] = np.log2(E[st][sy]) \
                    + log2sum([prev[j] + np.log2(A[k][st]) for j, k in enumerate(STATES)])
    return log2sum(logp)

forward(symbols)

For $x$ defined as above, $\log{\left(P\left(x\right)\right)}$ is

In [ ]:
forward(x)

In both of these examples, $P\left(x, \pi\right) \leq P\left(x\right)$. This is true in general because
$$P\left(x\right) = \sum_{\pi^\prime}{P\left(x, \pi^\prime\right)} = \sum_{\pi^\prime\neq\pi}{P\left(x, \pi^\prime\right)} + P\left(x, \pi\right) \geq P\left(x, \pi\right)$$

We can use the log of the forward matrix $F$ to perform the first step of the stochastic traceback algorithm by noting that
$$P\left(\pi_L = k \mid x\right) = \frac{f_k\left(L\right)}{P\left(x\right)} = \exp{\left(F_k\left(L\right) - \log{\left(P\left(x\right)\right)}\right)}$$

# Question 2

We can simulate trees on $n$ taxa according to the Yule model with branching parameter $\lambda$ by using the point process formalisation described by Tanja Gernhard (2008. The conditioned reconstructed process. _Journal of Theoretical Biology_, 253.4. pp. 769–778. doi:[`10.1016/j.jtbi.2008.04.005`](http://doi.org/10.1016/j.jtbi.2008.04.005)).

First we sample the time of origin $t$ by drawing $u$ uniformly at random from the interval $\left[0,1\right)$ and transforming it by
$$Q^{-1}_{or}\left(u\right) = - \frac{\log{\left(1 - \sqrt[n]{u}\right)}}{\lambda}$$
where $Q_{or}\left(t \mid n\right)$ is given in corollary 3.3.

In [ ]:
def random_t(n, lambd):
    return - np.log(1 - np.power(np.random.random(), 1/n)) / lambd

Then we sample $n-1$ speciation times $s$ by drawing $u$ uniformly at random from the the interval $\left[0,1\right)$ and transforming it by
$$F^{-1}\left(u\right) = - \frac{\log{\left(1 - \left(1 - \exp{\left(-\lambda t\right)}\right) u \right)}}{\lambda}$$
for each, where $F\left(s \mid t\right)$ is given in section 2.1.1.

In [ ]:
def random_s(n, lambd):
    t = random_t(n, lambd)
    s = - np.log(1 - (1 - np.exp(-lambd * t)) * np.random.random(n-1)) / lambd
    s.sort() # Most recent to oldest speciation times
    return s

To simulate a tree, we create a parent node with the next speciation time and assign it two children chosen uniformly at random from the available nodes. Then we remove these children from the available nodes and add the parent. We repeat this procedure $n-1$ times until there is only one available node, the root of the tree.

In [ ]:
def simulate_tree(n, lambd):
    nodes = []
    for i in map(str, range(1, n+1)):
        node = Node(i)
        node.set_height(0)
        nodes.append(node)
    for s in random_s(n, lambd):
        node = Node()
        i, j = np.random.choice(nodes, replace=False, size=2)
        node.set_height(s)
        node.add_child(i)
        node.add_child(j)
        nodes.remove(i)
        nodes.remove(j)
        nodes.append(node)
    return Tree(nodes[0])

Here we simulate $1000$ trees with $n = 10$ and $\lambda = 1$ and compute their mean height.

In [ ]:
np.mean([simulate_tree(10, 1).get_root().get_height() for _ in range(1000)])

This statistic agrees with the theoretical mean $1.93$.

To keep this notebook self-contained, I implement my own tree plotting function below.

In [ ]:
def compute_node_xy(node, counter=it.count()):
    node.x = node.get_height()
    if node.is_leaf():
        node.y = next(counter)
    else:
        children = node.get_children()
        for child in children:
            compute_node_xy(child, counter)
        node.y = np.mean([c.y for c in children])

def plot_node(node):
    if node.is_leaf():
        plt.text(node.x, node.y, ' ' + node.get_label(),
                 {'ha':'left', 'va':'center'})
    else:
        children = node.get_children()
        plt.plot([node.x] * 2, [min(c.y for c in children),
                                max(c.y for c in children)], 'k')
        for child in children:
            plt.plot([node.x, child.x], [child.y] * 2, 'k')
            plot_node(child)

def plot_tree(tree):
    root = tree.get_root()
    compute_node_xy(root)
    plt.plot([root.x, root.x + root.x/16], [root.y] * 2, 'k')
    plot_node(root)
    lc = tree.get_leaf_count()
    plt.ylim(- lc / 16, 17/16 * lc - 1)
    axes = plt.gca()
    axes.invert_xaxis()
    axes.yaxis.set_visible(False)
    axes.set_frame_on(False)
    axes.grid()

Here we simulate and plot a tree with $n = 10$ and $\lambda = 1$.

In [ ]:
tree = simulate_tree(10, 1)
plot_tree(tree)

Next, we simulate sequences down the tree according to the Jukes-Cantor model. At the root node, we draw a random sequence of length $L$ according to the base equilibrium frequencies.

In [ ]:
BASES = ('A', 'C', 'G', 'T')
def random_sequence(length):
    return ''.join(np.random.choice(BASES, size=length))

The sequence at a non-root node simulated by mutating its parent node's sequence for the time given by the difference in their heights. Over time $t$ a nucleotide $x$ in a sequence $S$ mutates with rate $\mu$ to each of the other bases with probability
$$p_{i \neq j} = \frac{1}{4} - \frac{1}{4} \exp{\left(-\frac{4}{3} \mu t\right)}$$
or stays the same with probability
$$p_{i = j} = \frac{1}{4} + \frac{3}{4} \exp{\left(-\frac{4}{3} \mu t\right)}$$

In [ ]:
def mutate_sequence(S, mu, t):
    ieqj = 1/4 + 3/4 * np.exp(-4/3 * mu * t)
    ineqj = 1/4 - 1/4 * np.exp(-4/3 * mu * t)
    p = {b : [ieqj if c == b else ineqj for c in BASES] for b in BASES}
    return ''.join(np.random.choice(BASES, p=p[x]) for x in S)

Lastly, the simulation recurses for each of a non-leaf node's children.

In [ ]:
def simulate_sequences(node, mu, length=None):
    if length:
        node = node.get_root()
        node.set_sequence(random_sequence(length))
    if not node.is_root():
        parent = node.get_parent()
        t = parent.get_height() - node.get_height()
        S = mutate_sequence(parent.get_sequence(), mu, t)
        node.set_sequence(S)
    if not node.is_leaf():
        for child in node.get_children():
            simulate_sequences(child, mu)

Here, we simulate sequences of length $L = 20$ with mutation rate $\mu = 0.5$ along our previously simulated tree.

In [ ]:
simulate_sequences(tree, 0.5, 20)

def get_sequences(tree):
    return sorted(tree.get_leaves(), key=lambda l: int(l.get_label()))
sequences = get_sequences(tree)

for seq in sequences:
    print('{:<3}: {}'.format(seq.get_label(), seq.get_sequence()))

The Jukes-Cantor distance $d_{xy}$ between two sequences $x$ and $y$ is given by
$$d_{xy} = -\frac{3}{4} \log{\left(1 - \frac{4}{3}f_{xy}\right)}$$
where $f_{xy}$ is the fraction of differing sites between $x$ and $y$ and is given by
$$f_{xy} = \min{\left(\frac{D_{xy}}{L}, \frac{3}{4} - \frac{1}{L}\right)}$$
where $D_{xy}$ is the Hamming distance between $x$ and $y$ and $L$ is the length of $x$.

In [ ]:
def D(x, y):
    return sum(a != b for a, b in zip(x, y))

def f(x, y):
    L = len(x)
    return min(D(x,y) / L, 3/4 - 1/L)

def d(x, y):
    return - 3/4 * np.log(1 - 4/3 * f(x, y))

We can find the distance matrix for a set of sequences by computing $d_{xy}$ for all pairs $x$ and $y$.

In [ ]:
def distance_matrix(sequences):
    sequences = list(map(Node.get_sequence, sequences))
    return [[d(x, y) for y in sequences] for x in sequences]

def distance_matrix_file(sequences, filename):
    d = distance_matrix(sequences)
    with open(filename, 'w') as file:
        for seq, row in zip(sequences, d):
            row = [seq.get_label()] + list(map(str, row))
            print(','.join(row), file=file)

Below is the distance matrix for our previously simulated sequences.

In [ ]:
for row in distance_matrix(sequences):
    print('  '.join(map('{:.3f}'.format, [x+0 for x in row])))

Here, we compare the quality of UPGMA reconstruction of the previously simulated tree for simulated sequences of lengths $L \in \left\{20, 50, 200\right\}$ mutating with rate $\mu = 0.2$. First we create the distance matrices.

In [ ]:
for L in (20, 50, 200):
    simulate_sequences(tree, 0.2, L)
    sequences = get_sequences(tree)
    distance_matrix_file(sequences, 'distL{}.csv'.format(L))

Then we run the provided script `doUPGMA.R` and view the reconstructed trees.

In [ ]:
subprocess.call(['Rscript', 'doUPGMA.R'])
output = 'UPGMAtree_%d.pdf'
subprocess.call(['pdftk', 'UPGMAtrees.pdf', 'burst', 'output', output])
for i in range(1, 4):
    with open(output % i, 'rb') as pdf:
        display_pdf(pdf.read(), raw=True)

In general, the quality of the reconstruction improves as the sequence length increases. In particular, the accuracy of the node heights improves substantially when considering more sequence data (it is important to note that the reconstructed heights $\mathbf{\hat{h}}$ are not expected to exactly match the true heights $\mathbf{h}$ but should be proportional such that $\mathbf{\hat{h}} = 2\mu\mathbf{h} = 0.4\mathbf{h}$). However, there are often still errors in the reconstructed topology even when using sequences of length 200, especially in regions of the tree with old divergences.